## Load Routines for Printing Matrices

In [1]:
include("printmat.jl")

println4Ps (generic function with 1 method)

# Return Calculations

From prices (measured after dividends) $P_t$ and dividends $D_t$ the return of holding the asset between $t-1$ and $t$ is

$
R_t = (P_t+D_t)/P_{t-1} - 1
$

In [2]:
P = [100;108;109]           #price series, after dividends
D = [0;2;0]     
t = 1:length(P)

R = (P[2:end] + D[2:end])./P[1:end-1] - 1
println("period and returns in %")
printmat([t[2:end] R*100])

period and returns in %
     2.000    10.000
     3.000     0.926



# Portfolio Return: Definition, Expected Value and Variance

With portfolio weights in the vector $w$, the portfolio return, the expected portfolio return and the portfolio variance can be computed as

$R_p = w'R$, $ER_p = w'\mu$ and $Var(R_p) = w'\Sigma w$

In [3]:
w = [0.8;0.2]
R = [10;5]/100          #returns of asset 1 and 2
Rp = w'R
println("portfolio weights: ")
printmat(w)
println("Returns of assets: ")
printmat(R)
printlnPs("Portfolio return: ",Rp)

portfolio weights: 
     0.800
     0.200

Returns of assets: 
     0.100
     0.050

Portfolio return:      0.090


In [4]:
μ = [9;6]/100                    #\mu and tab to get this
Σ = [256 96;
     96 144]/100^2

println("expected returns*100: ")
printmat(μ*100)

println("covariance matrix*100^2:")
printmat(Σ*100^2)

expected returns*100: 
     9.000
     6.000

covariance matrix*100^2:
   256.000    96.000
    96.000   144.000



In [5]:
ERp = w'μ
VarRp = w'Σ*w

println("expected portfolio return, variance and std of portfolio return: ")
printmat([ERp VarRp sqrt.(VarRp)])

expected portfolio return, variance and std of portfolio return: 
     0.084     0.020     0.142



In [6]:
Σb = [256 -96;
      -96 144]/100^2

                                        #()[1] to extract a scalar, not an 1x1 matrix
printlnPs("portfolio std when the assets are negatively correlated:",sqrt((w'Σb*w)[1]))

portfolio std when the assets are negatively correlated:     0.118


# Matrix Appendix

In [7]:
A = [1 3;3 4]
c = 10
printmat(A + c)
printmat(A*c)

        11        13
        13        14

        10        30
        30        40



In [8]:
A = [10;11]
B = [2;5]
printmat(A-B)
printmat([1 3;3 4] + [1 2;3 -2])

         8
         6

         2         5
         6         2



In [9]:
printmat([10;11]')
printmat([1 2 3;4 5 6]')

        10        11

         1         4
         2         5
         3         6



In [10]:
printmat([1 3;3 4]*[1 2;3 -2])
printmat([1 3;3 4]*[2;5])

        10        -4
        15        -2

        17
        26



In [11]:
printmat([10;11]'*[2;5])
printmat([10;11]*[2;5]')
printmat([10;11]'*[1 3;3 4]*[10;11])

        75

        20        50
        22        55

      1244



In [12]:
printmat(inv([1 3;3 4]))
printmat(inv([1 3;3 4])*[1 3;3 4])

    -0.800     0.600
     0.600    -0.200

     1.000    -0.000
     0.000     1.000

